In [60]:
import os
import re
import sys
import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np

# Image related imports
import av # TODO: Remove this after porting to PIL
from PIL import Image
# from resizeimage import resizeimage
from skimage.feature import hog
from skimage import data, color

import matplotlib.pyplot as plt
import pickle

import seaborn as sns; sns.set()
%matplotlib inline

In [61]:
path_to_video = 'test.mp4'
data_dir = '../data'
images_dir = os.path.join(data_dir, 'frames_tmp/')
model_dir = '../imagenet/classify_image_graph_def.pb'
results_dir = '../results/'
cnn_diff_dir = os.path.join(results_dir, 'cnn_diff_tmp')
pixel_diff_dir = os.path.join(results_dir, 'pixel_diff_tmp')
out_dir = os.path.join(results_dir, 'combined_frames_tmp')

In [62]:
def frame_extractor(path_to_video, path_to_frames):
    container = av.open(path_to_video)
    video = next(s for s in container.streams if s.type == b'video')
    for packet in container.demux(video):
        for frame in packet.decode():
            frame.to_image().save(path_to_frames + '/frame-%06d.jpg' % frame.index)

In [69]:
class FeatureProcessor:
    def __init__(self, model_path, path_to_frames, sampling_rate=10, saliency_threshold=0.8):
        self.model_path = model_path
        self.images_path = sorted([path_to_frames + f for f in os.listdir(path_to_frames) if re.search('jpg|JPG', f)])
        self.sampling_rate = sampling_rate
        self.saliency_threshold = saliency_threshold
        
    def create_graph(self):
        with gfile.FastGFile(self.model_path, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            _ = tf.import_graph_def(graph_def, name='')
    
    def get_layer_list(self):
        self.create_graph()
        with tf.Session() as sess:
            operation_list = sess.graph.get_operations()
            for op in operation_list:
                print op.name
                print op.values().get_shape()

    # TODO: Experiment with other layers.            
    def extract_features(self, layer_name='pool_3:0', nb_features=2048):
        features = np.zeros((len(self.images_path), nb_features))
        labels = []
        self.create_graph()
        with tf.Session() as sess:
            next_to_last_tensor = sess.graph.get_tensor_by_name(layer_name)
            for index, image in enumerate(self.images_path):
                if (index % 500 == 0 and index > 0):
                    print('Processing %s...' % (image))
                if not gfile.Exists(image):
                    tf.logging.fatal('File does not exist %s', image)
                    continue
                image_data = gfile.FastGFile(image, 'rb').read()
                predictions = sess.run(next_to_last_tensor, {'DecodeJpeg/contents:0': image_data})
                features[index, :] = np.squeeze(predictions)
        return features
    
    def compute_change_points(self, distances):
        change_points = np.where(distances > self.saliency_threshold)
        return change_points
    
    def get_pixel_differences(self):
        prev_feature = np.array(Image.open(self.images_path[0])).flatten()
        nb_features = len(prev_feature.flatten())
        distances = []
        for index, next_path in enumerate(self.images_path[1:]):
                if (index % 500 == 0 and index > 0):
                    print('Processing Distance for %s...' % (next_path))
                next_feature = np.array(Image.open(next_path)).flatten()
                distances.append(np.linalg.norm(next_feature - prev_feature))
                prev_feature = next_feature
        return distances
    
    @staticmethod
    def get_feature_differences(features):
        features_1 = features[:-1, :]
        features_2 = features[1:, :]
        saliency_matrix = features_2 - features_1
        
        differences = []
        for vector in saliency_matrix:
            differences.append(np.linalg.norm(vector))
        differences = np.asarray(differences)
        return differences, saliency_matrix
    
    @staticmethod
    def save_difference_plots(differences, path, color):
        # 5 = 120 pixels
        differences = differences / max(differences)
        plt.rcParams['figure.figsize'] = (11.25, 3.75) 
        plt.figure()
        num_frames = len(differences)
        for i in range(num_frames):
            plot_file = os.path.join(path, '%06d.png' % i)
            plt.plot(differences[:i+1], color=color)
            plt.xlim([0, num_frames])
            plt.ylim([0, 1])
            plt.savefig(plot_file)
         
    @staticmethod
    def combine_images():
        # plt.rcParams['figure.figsize'] = (10, 5)
        image_paths = os.listdir(images_dir)
        num_images = len(image_paths)
        for i in range(num_images - 1):
            if i > 2:
                break
            if i % 500 == 0:
                print('Now processing', i, 'out of', num_images)
            im_file = os.path.join(images_dir, image_paths[i])
            cnn_diff_file = os.path.join(cnn_diff_dir, '%06d.png' % i)
            pixel_diff_file = os.path.join(pixel_diff_dir, '%06d.png' % i)
            out_file = os.path.join(out_dir, '%06d.png' % i)
            im = np.asarray(Image.open(im_file))
            cnn_diff_im = np.asarray(Image.open(cnn_diff_file))
            pixel_diff_im = np.asarray(Image.open(pixel_diff_file))
            diff_im = np.vstack([cnn_diff_im, pixel_diff_im])
            combined = np.hstack([im, diff_im])
            plt.imshow(combined)
            imsave(os.path.join(out_dir, '%06d.png' % i), combined)    
    
    @staticmethod                       
    def get_smoothed_list(list_to_smooth, window_size = 10):
        num_elements = len(list_to_smooth)
        smoothed_list = []
        cur_sum = sum(list_to_smooth[:window_size])
        smoothed_list.append(cur_sum * 1.0 / window_size)
        for i in xrange(window_size, num_elements):
            cur_sum -= list_to_smooth[i-window_size]
            cur_sum += list_to_smooth[i]
            smoothed_list.append(cur_sum * 1.0 / window_size)
        return smoothed_list
    
    @staticmethod
    def plot_differences(feature_differences, pixel_differences):
        feature_differences = feature_differences / max(feature_differences)
        pixel_differences = pixel_differences / max(pixel_differences)
        plt.plot(feature_differences, label='Feature Differences')
        plt.plot(pixel_differences, label='Pixel Differences')

In [ ]:
if __name__ == "__main__":
    #frame_extractor(path_to_video, images_dir)
    feature_processor = FeatureProcessor(model_dir, images_dir)
    feature_processor.create_graph()
    features = feature_processor.extract_features()
    cnn_differences, saliency_matrix = feature_processor.get_feature_differences(features)
    pixel_differences = feature_processor.get_pixel_differences()
    #feature_processor.plot_differences(cnn_differences, pixel_differences)
    feature_processor.plot_differences(get_smoothed_list(cnn_differences), get_smoothed_list(pixel_differences))
    feature_processor.save_difference_plots(cnn_differences, cnn_diff_dir, '#ff6666')
    feature_processor.save_difference_plots(pixel_differences, pixel_diff_dir, '#79cdcd')
    feature_processor.combine_images()